In [ ]:
from scipy.interpolate import griddata

In [ ]:
def create_img2(tsne_plots, samples, n_grid = 128, 
               verbose = False, 
               verbose_sample = False, 
               verbose_marker = False):
    """create image from t-SNE plot
    
    Position arguments:
    tsne_plots --- numpy array; (num_samples_tot, num_cells_tot, 2)
    samples    --- numpy array; (num_samples_tot, num_cells_tot, num_features)
    
    Keyword arguments:
    n_grid  --- the dimension of image (n_grid x n_grid)
    verbose --- print out the running process
    """
    
    if (verbose):
        print("Create images from t-SNE plot...")
    
    # check the dimension
    checked_value, error_message = check_dimension(samples)
    assert (checked_value == 3), error_message
    
    # initialization
    num_samples_tot = samples.shape[0]
    result_img = []
    
    # iterate though each samples
    for idx_sample in range(num_samples_tot):
        if (verbose_sample):
            print("\tPrepare image of the", idx_sample, "sample")
        
        # initialization in each loop
        sample = samples[idx_sample]
        num_cells_tot = sample.shape[0]
        num_features  = sample.shape[1]
        
        # get x, y coordinate of a plot
        tsne_plot = tsne_plots[idx_sample]
        x = tsne_plot[:, 0]
        y = tsne_plot[:, 1]
        
        # generate a grid
        x_c = np.linspace(min(x), max(x), n_grid)
        y_c = np.linspace(min(y), max(y), n_grid)
        x_c, y_c = np.meshgrid(x_c, y_c)
        
        # each feature is a layer/channel for the plot
        # to get each layer, perform interpolation to convert tSNE plot in a image
        img = []
        for idx_feature in range(num_features):
            
            if (verbose_marker):
                print("\t\tinterpolating the", idx_feature, "feature")
            
            # interpolation
            z = sample[:, idx_feature]
            rbfi = Rbf(x, y, z, function='cubic', smooth=1)
            #griddata(points, values, (grid_x, grid_y), method='linear')
            #z_c = griddata(tsne_plot, z, (x_c, y_c), method='linear')
            
            
            # store into a list "img"
            z_c = rbfi(x_c, y_c)
            img.append(z_c)
            
        # normalize & arrange the interpolated feature values    
        img = np.array(img)
        img = (img - np.min(img)) / (np.max(img) - np.min(img))
        
        # append each interpolated result to the result
        result_img.append(img)
        
    # standardize images
    result_img = [(img - np.min(img)) / (np.max(img) - np.min(img)) for img in result_img]
    result_img = np.array(result_img)
    
    if (verbose):
        print("...Finish")
        
    return result_img

In [ ]:
def func(x, y):
     return x*(1-x)*np.cos(4*np.pi*x) * np.sin(4*np.pi*y**2)**2

x_c = np.linspace(-1, 2, 100)
y_c = np.linspace(-10, 2, 100)
#grid_x, grid_y = np.mgrid[0:1:100j, 0:1:200j]
grid_x, grid_y = np.meshgrid(x_c, y_c)
points = np.random.rand(1000, 2)
values = func(points[:,0], points[:,1])
grid_z0 = griddata(points, values, (grid_x, grid_y), method='nearest')